<h1>Exploration dataset by Grégoire Corluy</h1>

Python version: 3.10 </br>
Author: Grégoire Corlùy

<h4>Load the data</h4>

In [5]:
import pandas as pd

In [6]:
dataset = pd.read_csv('../datasets/train.csv')
print(f"Size of the dataset is {dataset.shape}")

Size of the dataset is (5044, 39)


Explore the data

In [7]:
#print the labels of the columns
print(dataset.columns)

Index(['Gender', 'Age', 'Connect_Date', 'L_O_S', 'Dropped_Calls', 'tariff',
       'Handset', 'Peak_calls_Sum', 'Peak_mins_Sum', 'OffPeak_calls_Sum',
       'OffPeak_mins_Sum', 'Weekend_calls_Sum', 'Weekend_mins_Sum',
       'International_mins_Sum', 'Nat_call_cost_Sum', 'AvePeak', 'AveOffPeak',
       'AveWeekend', 'National_calls', 'National mins', 'AveNational',
       'All_calls_mins', 'Dropped_calls_ratio', 'Usage_Band', 'Mins_charge',
       'call_cost_per_min', 'actual call cost', 'Total_call_cost',
       'Total_Cost', 'Tariff_OK', 'average cost min', 'Peak ratio',
       'OffPeak ratio', 'Weekend ratio', 'Nat-InterNat Ratio',
       'high Dropped calls', 'No Usage', 'target', 'id'],
      dtype='object')


Number of positive and negative targets

In [8]:
print(dataset.head)

<bound method NDFrame.head of      Gender   Age Connect_Date      L_O_S  Dropped_Calls    tariff Handset  \
0         F  50.0     20/05/98  29.200000            2.0  Play 100   BS210   
1         M  25.0     16/12/96  46.533333            1.0   CAT 100  ASAD90   
2         F  46.0     27/08/97  38.066667            1.0    CAT 50    WC95   
3         F  59.0     13/07/98  27.400000            1.0    CAT 50   BS110   
4         F  25.0     08/01/99  21.433333            1.0  Play 300    WC95   
...     ...   ...          ...        ...            ...       ...     ...   
5039      F  16.0     15/09/97  37.433333            2.0   CAT 200  ASAD90   
5040      F  29.0     28/09/96  49.166667            1.0   CAT 100   CAS30   
5041      M  23.0     08/06/99  16.400000            0.0   CAT 200   BS110   
5042      M  46.0     01/03/98  31.866667            2.0  Play 100     S50   
5043      M  61.0     12/11/98  23.333333            0.0   CAT 100     S50   

      Peak_calls_Sum  Peak_mins_S

In [12]:
#count number of ones and zeros as target
nbr_ones = dataset["target"].value_counts()[1]
nbr_zeros = dataset["target"].value_counts()[0]

print(f"number of ones as target: {nbr_ones}")
print(f"number of zeros as target: {nbr_zeros}")

#Check total number of rows is equal to number of zeros and ones
missing_targets = (nbr_ones+nbr_zeros != dataset.shape[0])
Missing_question = "Yes" if missing_targets else "No"

print()
print(f"Missing values in target? {Missing_question}")

number of ones as target: 745
number of zeros as target: 4299

Missing values in target? No


This unbalance has to be taken into account during the train-validation split and during the training.</br>
Can the used model handle this unbalance or will it biased?

Analyze all the other features:</br>
Check for missing values, outliers, distribution of the data

In [13]:
#Gender
print(dataset["Gender"].head)

<bound method NDFrame.head of 0       F
1       M
2       F
3       F
4       F
       ..
5039    F
5040    F
5041    M
5042    M
5043    M
Name: Gender, Length: 5044, dtype: object>


In [15]:
#check for missing values
nbr_Male = dataset["Gender"].value_counts()["M"]
nbr_Female = dataset["Gender"].value_counts()["F"]

print(f"number of males as target: {nbr_Male}")
print(f"number of females as target: {nbr_Female}")

missing_gender= (nbr_Male+nbr_Female != dataset.shape[0])
Missing_question = "Yes" if missing_gender else "No"

print()
print(f"Missing values in gender? {Missing_question}")

number of males as target: 2447
number of females as target: 2597

Missing values in gender? No


Number of males and females is more or less balanced